# Synergy Logistics

**Synergy Logistics** es una empresa dedicada a la intermediación de servicios de importación y exportación de diferentes productos. Actualmente la empresa cuenta con una base de datos que refleja las rutas más importantes que opera desde el año 2015, con su respectivo origen y destino, año, producto, modo de transporte y valor total. Nuestro propósito, es que a partir de estos datos se genere un análisis que sirva de la base para la estructuración de su estrategia operativa.

Los datos que nos fueron proporcionados por Synergy Logistics son los siguientes.

In [79]:
import pandas as pd
import numpy as np
df = pd.read_csv("synergy_logistics_database.csv")
df.head()

,register_id,direction,origin,destination,year,date,product,transport_mode,company_name,total_value
0,1,Exports,Japan,China,2015,31/01/15,Cars,Sea,Honda,33000000
1,2,Exports,Japan,China,2015,01/02/15,Cars,Sea,Honda,16000000
2,3,Exports,Japan,China,2015,02/02/15,Cars,Sea,Honda,29000000
3,4,Exports,Japan,China,2015,03/02/15,Cars,Sea,Honda,14000000
4,5,Exports,Japan,China,2015,04/02/15,Cars,Sea,Honda,17000000


Con el fin de crear una estrategia operativa para el año 2021, tenemos analizar los datos proporcionados para obtener 3 enfoques:
- Rutas de importación y exportación
- Medios de transporte usados
- Valor de las importaciones y exportaciones

## 10 rutas con más demandas

### Rutas de importaciones

In [82]:
rutas_importaciones = df.loc[df["direction"] == "Imports"]
rutas_importaciones = rutas_importaciones.groupby(['direction', 'origin', 'destination'], group_keys=False).count()
rutas_importaciones.reset_index()
rutas_importaciones.drop(['register_id', 'year', 'date', 'product', 'transport_mode', 'company_name'], axis=1, inplace=True)
rutas_importaciones.rename(columns={'total_value': 'count'}, inplace=True)
rutas_importaciones.sort_values('count', ascending=False).head(10)

count
direction origin    destination                
Imports   Singapore Thailand                273
          Germany   China                   233
          China     Japan                   210
          Japan     Mexico                  206
          China     Thailand                200
          Malaysia  Thailand                195
          Spain     Germany                 142
          Mexico    USA                     122
          China     United Arab Emirates    114
          Brazil    China                   113

### Rutas exportaciones

In [83]:
rutas_exportaciones = df.loc[df["direction"] == "Exports"]
rutas_exportaciones = rutas_exportaciones.groupby(['direction', 'origin', 'destination'], group_keys=False).count()
rutas_exportaciones.reset_index()
rutas_exportaciones.drop(['register_id', 'year', 'date', 'product', 'transport_mode', 'company_name'], axis=1, inplace=True)
rutas_exportaciones.rename(columns={'total_value': 'count'}, inplace=True)
rutas_exportaciones.sort_values('count', ascending=False).head(10)

count
direction origin      destination       
Exports   South Korea Vietnam        497
          Netherlands Belgium        437
          USA         Netherlands    436
          China       Mexico         330
          Japan       Brazil         306
          Germany     France         299
          South Korea Japan          279
          Australia   Singapore      273
          Canada      Mexico         261
          China       Spain          250

### Rutas importaciones y exportaciones

In [163]:
rutas = df.groupby(['direction', 'origin', 'destination'], group_keys=False).count()
rutas.reset_index()
rutas.drop(['register_id', 'year', 'date', 'product', 'transport_mode', 'company_name'], axis=1, inplace=True)
rutas.rename(columns={'total_value': 'count'}, inplace=True)
rutas.sort_values('count', ascending=False).head(10)

count
direction origin      destination       
Exports   South Korea Vietnam        497
          Netherlands Belgium        437
          USA         Netherlands    436
          China       Mexico         330
          Japan       Brazil         306
          Germany     France         299
          South Korea Japan          279
          Australia   Singapore      273
Imports   Singapore   Thailand       273
Exports   Canada      Mexico         261

### Conclusión

Si comparamos esta tabla con la de la suma de los montos

In [169]:
rutas = df.groupby(['direction', 'origin', 'destination'], group_keys=False).sum()
rutas.reset_index()
rutas.drop(['register_id', 'year'], axis=1, inplace=True)
rutas.sort_values('total_value', ascending=False).head(10)

total_value
direction origin      destination                
Exports   China       Mexico          12250000000
          Canada      Mexico           8450000000
          South Korea Vietnam          6877007000
          France      Belgium          5538069000
                      United Kingdom   5427000000
          China       South Korea      4790000000
          USA         Mexico           4710000000
          South Korea Japan            4594000000
          Germany     Italy            4541000000
          China       Germany          4090000000

No se recomienda elegir las rutas con más demandas, ya que no son estas precisamente las que más monto generan.

## Medios de transporte más importantes

In [12]:
transportes = df.groupby(['transport_mode'], group_keys=False).sum()
transportes.drop(['register_id', 'year'], axis=1, inplace=True)
transportes.sort_values('total_value', ascending=False)

,total_value
transport_mode,
Sea,100530622000
Rail,43628043000
Air,38262147000
Road,33270486000


### Conclusión

Los 3 medios de transporte más importantes para Synergy Logistics son: Mar, Ferrocarril y Aire.
El que genera menos valor es por carretera, por lo que pueden reducir este.

Se recomienda encontrar maneras de completar las rutas por carretera a través de los otros medios de transporte para no perder totalmente las importaciones y exportaciones en esas rutas.

## 80% de las ganancias

In [92]:
rutas = df.groupby(['direction', 'origin', 'destination'], group_keys=False).sum()
gran_total = rutas['total_value'].sum()
rutas.reset_index()
rutas.drop(['register_id', 'year'], axis=1, inplace=True)
rutas.sort_values('total_value', ascending=False, inplace=True)

pd.set_option('display.float_format', '{:.10f}'.format)
rutas['porcentaje'] = rutas['total_value'].cumsum() / gran_total
rutas_80 = rutas.loc[rutas['porcentaje'] <= 0.8]
rutas_80

total_value   porcentaje
direction origin      destination                                   
Exports   China       Mexico                12250000000 0.0567941318
          Canada      Mexico                 8450000000 0.0959704921
          South Korea Vietnam                6877007000 0.1278540546
          France      Belgium                5538069000 0.1535299584
                      United Kingdom         5427000000 0.1786909178
...                                                 ...          ...
Imports   Mexico      Japan                  1143000000 0.7776517159
          USA         India                  1133000000 0.7829045936
Exports   Spain       Russia                 1085000000 0.7879349310
          India       United Arab Emirates   1037000000 0.7927427281
          USA         Netherlands            1032187000 0.7975282109

[61 rows x 2 columns]

Elegimos solos los paises, primero por origin y también por destination.

In [99]:
count_origin = rutas_80.groupby(['origin']).count()
count_origin.drop(['total_value'], axis=1, inplace=True)
count_origin.rename(columns={'porcentaje': 'count'}, inplace=True)
count_origin

,count
origin,
Belgium,1
Canada,1
China,11
France,5
Germany,5
India,1
Italy,1
Japan,7
Malaysia,1


In [100]:
count_origin.index.tolist()

['Belgium',
 'Canada',
 'China',
 'France',
 'Germany',
 'India',
 'Italy',
 'Japan',
 'Malaysia',
 'Mexico',
 'Netherlands',
 'Russia',
 'Singapore',
 'South Korea',
 'Spain',
 'USA',
 'United Kingdom']

In [95]:
#count_destination = rutas_80.groupby(['destination']).count()
#count_destination.drop(['total_value'], axis=1, inplace=True)
#count_destination.rename(columns={'porcentaje': 'count'}, inplace=True)
#count_destination

In [94]:
#count_paises = count_origin.add(count_destination, fill_value=0)
#count_paises['count'] = count_paises['count'].astype(int)
#count_paises.index.tolist()

Veamos los paises presentes en el 100% de las importaciones y exportaciones.

In [103]:
count_origin_100 = rutas.groupby(['origin']).count().reset_index()
count_origin_100.drop(['total_value'], axis=1, inplace=True)
count_origin_100.rename(columns={'porcentaje': 'count'}, inplace=True)
count_origin_100

#count_destination_100 = rutas.groupby(['destination']).count()
#count_destination_100.drop(['total_value'], axis=1, inplace=True)
#count_destination_100.rename(columns={'porcentaje': 'count'}, inplace=True)

#count_paises_100 = count_origin.add(count_destination, fill_value=0)
#count_paises_100['count'] = count_paises['count'].astype(int)
#count_paises_100.index.tolist()

,origin,count
0,Australia,7
1,Austria,6
2,Belgium,4
3,Brazil,7
4,Canada,7
5,China,16
6,France,13
7,Germany,13
8,India,6
9,Italy,14


## Conclusión general

A Synergy Logistics le conviene implementar dos estrategias: basada en las 